In [ ]:
# uncomment and pip install if not already done
#!pip install spotipy


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# all the imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from vinylitics.params import *

## set up spotify credentials

Go to Spotify Dashboard  
https://developer.spotify.com/dashboard  
  
Create an account if need be and login  
  
Click on your user name top right, then dashboard, then create app:  

1.	Click “Create an App”  
2.	Fill in the required details:  
	•	App Name: Choose any name (e.g., “Spotify Track Finder”)  
	•	App Description: Short description (optional)  
	•	Redirect URI: not useful but needed, fill in a placeholder like 'http://localhost:8888/callback'
3.	Accept the terms and conditions and click “Create”
  
Your app will appear on your dashboard, go to settings and get your Client ID and Client Secret



In [4]:
# authenticate to spotify account
sp = spotipy.Spotify(
    auth_manager= SpotifyClientCredentials(
        client_id=SPOTIFY_CLIENT_ID
        , client_secret=SPOTIFY_CLIENT_SECRET
        )
    )

In [5]:
# function that outputs track_id from spotify API

def get_track_id_top_match(track_name, artist_name= None):
    """Search for a track on Spotify and return its track ID."""
    query = f"track:{track_name}"
    if artist_name:
        query += f" artist:{artist_name}"

    result = sp.search(q= query, type= "track", limit=1)  # get the top match

    if result['tracks']['items']:
        track = result['tracks']['items'][0]
        track_id = track['id']
        track_name = track['name']
        track_artist = track['artists'][0]['name']

        return {
            "track_id": track_id,
            "track_name": track_name,
            "artist_name": track_artist,
            "spotify_url": f"https://open.spotify.com/track/{track_id}"
        }
    else:
        return {"error": "Track not found"}

In [11]:
# test example
track_info = get_track_id_top_match("christmas")
print(track_info)

{'track_id': '0GsNzZNqLPrNnJswoQOCtO', 'track_name': 'Christmas Bae', 'artist_name': 'Yade Lauren', 'spotify_url': 'https://open.spotify.com/track/0GsNzZNqLPrNnJswoQOCtO'}


In [ ]:
# function that solves for several matches, prints back the top 10 matches and asks user to select which song they meant
# WARNING - this does not work on jupyter notebook and needs to be added to a python script in order for the input function to work (otherwise it runes endlessly)

def get_track_id_top_10_match(track_name, artist_name= None):

    """
    search for a track on spotify, display multiple matches, and let the user select the correct track
    """

    query = f"track:{track_name}"
    if artist_name:
        query += f" artist:{artist_name}"

    result = sp.search(q= query, type= "track", limit= 10)  # get up to 10 matches

    tracks = result['tracks']['items']
    if not tracks:
        print("no tracks found :-(")
        return None

    # display the matches
    print("\nseveral possible matches found:")
    for idx, track in enumerate(tracks):
        print(f"{idx + 1}. {track['name']} - {', '.join(artist['name'] for artist in track['artists'])}")

    # get user selection
    while True:
        try:
            choice = int(input(f"\nwhich song did you mean (1-{len(tracks)}): ")) - 1
            if 0 <= choice < len(tracks):
                selected_track = tracks[choice]
                track_id = selected_track['id']
                track_name = selected_track['name']
                artist_name = ", ".join(artist['name'] for artist in selected_track['artists'])

                return {
                    "track_id": track_id,
                    "track_name": track_name,
                    "artist_name": artist_name,
                    "spotify_url": f"https://open.spotify.com/track/{track_id}"
                }
            else:
                print("invalid selection, please choose a number from the list")
        except ValueError:
            print("invalid input, please enter a number")

In [15]:
# test example
track_info = get_track_id_top_10_match("hello")


several possible matches found:
1. Hello - Adele
2. Hello Baby (feat. Julia Church) - Cinimin, Julia Church
3. Hello Miss Johnson - Jack Harlow
4. Hello - Moto/Spory - Kukon, Magiera
5. Hello - Lionel Richie
6. 멋지게 인사하는 법 (Hello Tutorial) (Feat. 슬기 (SEULGI) of Red Velvet) - Zion.T, 슬기 (SEULGI)
7. Hello Joehoe - D-Double
8. Hello - Party Mix - Wajid Khan, Harchelle, Tim, Suzi Q, Ishq Bector, Merlin
9. Hello - Martin Solveig, Dragonette
10. Hello How Are You (ZDF Silvester-Hitparty 31.12.2003) - VOD - No Mercy
